In [1]:
!pip install transformers -q

In [13]:
#lets try and implement some models that have been built in hugging face transformers by importing the models

from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [12]:
sentence_pos = 'Abuja weather is good and awesome'
sentence_neg = 'The weather of abuja not good for the body'

In [14]:
classifier(sentence_pos), classifier(sentence_neg)

([{'label': 'POSITIVE', 'score': 0.9998857975006104}],
 [{'label': 'NEGATIVE', 'score': 0.9996659755706787}])

In [15]:
#importing some models for use

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertForSequenceClassification
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [17]:
inputs_pos, inputs_neg = tokenizer(sentence_pos), tokenizer(sentence_neg)

In [18]:
inputs_pos, inputs_neg

({'input_ids': [101, 8273, 3900, 4633, 2003, 2204, 1998, 12476, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 1996, 4633, 1997, 8273, 3900, 2025, 2204, 2005, 1996, 2303, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [20]:
#this result are tokens id . lets do and example below to see what we have..
#check this hugging face documentation page for tokenizers to be able to see what functions can be used.
#https://huggingface.co/docs/transformers/main_classes/tokenizer


tokenizer.convert_ids_to_tokens([4633, 2003, 2204, 1998, 8273, 3900])



['weather', 'is', 'good', 'and', 'abu', '##ja']

In [21]:
tokenizer.convert_ids_to_tokens([8273, 3900, 2025, 2204, 2005, 1996, 2303,])


#here you will see that abuja as a word is been split  into two tokens because the distilbert vocalbulary does not contain the token for abuja
#so it try to break it up to words it knows in a small way. so we can say it give prefrence to common words.

['abu', '##ja', 'not', 'good', 'for', 'the', 'body']

In [22]:
tokenizer.convert_tokens_to_string(['abu', '##ja', 'is', 'little'])

#we can see it brought back the abuja word as presented to it in the first place.

'abuja is little'

In [24]:
#tensors must be rectangular or square


pt_padd = tokenizer(
    [sentence_pos, sentence_neg],
    padding = True,
    truncation = True,
    max_length = 512,
    return_tensors = "pt"
)


In [25]:
pt_padd.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [26]:
pt_outputs = pt_model(**pt_padd)

In [28]:
pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.3416,  4.7361],
        [ 4.4219, -3.5822]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [30]:
from torch import nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)

In [18]:
pt_predictions

tensor([[1.1418e-04, 9.9989e-01],
        [9.8944e-01, 1.0565e-02]], grad_fn=<SoftmaxBackward0>)